
# 0 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

import os

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

def export_results(p2p, name):
    # open file in write mode
    with open(r'data_out/'+ name + '.txt', 'w') as f_export_init:
        for item in p2p:
            # write each item on a new line
            f_export_init.write("%s\n" % item)
        print('Exported' + name + 'point to point mapping \n')



# 1 - Data loading

In [ ]:
# Input folder name
inputFolder = 'PAIR_TEST'
# Concatenate folder name with the path
inputPath = '../../data/' + inputFolder + '/'
mesh1 = TriMesh(inputPath + 'target.off', area_normalize=True, center=True)
mesh2 = TriMesh(inputPath + 'source.off', area_normalize=True, center=True)

# 2- Visualization of eigen functions

In [ ]:
# By default does not use the intrinsic delaunay Laplacian
mesh2.process(k=100, intrinsic=False, verbose=True);

# set up a figure twice as wide as it is tall
i = 0
d = mp.subplot(mesh2.vertlist, mesh1.facelist, c=mesh2.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh2.vertlist, mesh1.facelist, c=mesh2.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

# 3 - Computing the landmarks based descriptor

**Displaying data with correspondences**

In [ ]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

loadLandmark = True
if loadLandmark:
    landmarks = np.loadtxt(inputPath +'landmarks.txt',dtype=int)[:10]  # loading N landmarks
    display(landmarks)

    landmarks_mesh1 = TriMesh(mesh1.vertlist[landmarks[:,0]])
    landmarks_mesh2 = TriMesh(mesh2.vertlist[landmarks[:,1]])

    print('Retrieved matching faces from CT & RGBD pointclouds')
    
else:
     landmarks = []

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)

if loadLandmark:
    color = c=np.random.rand(*landmarks_mesh1.vertlist.shape)
    p1.add_points(landmarks_mesh1.vertlist, c = color, shading={"point_size": 0.3})
    p2.add_points(landmarks_mesh2.vertlist, c = color, shading={"point_size": 0.3})


**Computing descriptors**

In [ ]:
verbose = True

model = FunctionalMapping(mesh1,mesh2)

n_ev = (25,25)  # Number of eigenvalues on source and Target
descr_type = 'HKS'
model.k1, model.k2 = n_ev

k_process = 200

use_lm = landmarks is not None and len(landmarks) > 0

# Compute the Laplacian spectrum
if verbose:
    print('\nComputing Laplacian spectrum')
model.mesh1.process(max(model.k1, k_process), verbose=verbose)
model.mesh2.process(max(model.k2, k_process), verbose=verbose)

if verbose:
    print('\nComputing descriptors')

# Extract landmarks indices
if use_lm:
    lmks1, lmks2 = model._get_lmks(landmarks, verbose=False)

In [ ]:
import pyFM.signatures as sg
n_descr = 5
subsample_step = 5

# Compute descriptors
if descr_type == 'HKS':
    if use_lm:
        lm_descr1 = sg.mesh_HKS(model.mesh1, n_descr,landmarks=lmks1, k=model.k1)  # (N1, p*n_descr)
        lm_descr2 = sg.mesh_HKS(model.mesh2, n_descr, landmarks=lmks2, k=model.k2)  # (N2, p*n_descr)

elif descr_type == 'WKS':
    if use_lm:
        lm_descr1 = sg.mesh_WKS(model.mesh1, n_descr, landmarks=lmks1, k=model.k1)  # (N1, p*n_descr)
        lm_descr2 = sg.mesh_WKS(model.mesh2, n_descr, landmarks=lmks2, k=model.k2)  # (N2, p*n_descr)
        
else:
    raise ValueError(f'Descriptor type "{descr_type}" not implemented')

## Subsample descriptors
#lm_descr1 = lm_descr1[:, np.arange(0, lm_descr1.shape[1], subsample_step)]
#lm_descr2 = lm_descr2[:, np.arange(0, lm_descr2.shape[1], subsample_step)]

# Normalize descriptors
#if verbose:
#    print('\tNormalizing descriptors')
#
#no1 = np.sqrt(model.mesh1.l2_sqnorm(lm_descr1))  
#no2 = np.sqrt(model.mesh2.l2_sqnorm(lm_descr2)) 
#
#model.descr1 /= no1[None, :]
#model.descr2 /= no2[None, :]
#
#if verbose:
#    n_lmks = np.asarray(landmarks).shape[0] if use_lm else 0
#    print(f'\n\t{model.descr1.shape[1]} out of {n_descr*(1+n_lmks)} possible descriptors kept')

In [ ]:
export_name_1 = 'lm_descr1'
export_name_2 = 'lm_descr2'

# Concatenate the descriptor type to the export name
if descr_type == 'HKS':

    export_name_1 = export_name_1 + '_HKS'
    export_name_2 = export_name_2 + '_HKS'
elif descr_type == 'WKS':
    export_name_1 = export_name_1 + '_WKS'
    export_name_2 = export_name_2 + '_WKS'

# Export the descriptors to a txt file
export_name_1 = export_name_1 + '.txt'
export_name_2 = export_name_2 + '.txt'

# Display export name
print(export_name_1)
print(export_name_2)

np.savetxt(export_name_1 , lm_descr1, delimiter=',')
np.savetxt(export_name_2 , lm_descr2, delimiter=',')



In [ ]:
process_params = {
    'n_ev': (25,25),  # Number of eigenvalues on source and Target
    'landmarks': landmarks,  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'HKS',  # WKS or HKS
}


model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);

**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [ ]:
fit_params = {
    'w_descr': 1e0, #scaling for the descriptor preservation term
    'w_lap': 1e-2, #scaling of the laplacian commutativity term
    'w_dcomm': 1e-1, #scaling of the multiplicative operator commutativity
    'w_orient': 0, #scaling of the orientation preservation term
    #'orient_reversing':True, #Whether to use the orientation reversing term instead of the orientation preservation one
    #'optinit':'zeros' #Initialization

}

model.fit(**fit_params, verbose=True)

**Visualizing the associated point to point map**

In [ ]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)